# AlphaEarth 64-D Embedding Probe

Sample the 64-dimensional AlphaEarth embedding vector from the Brahmanbaria mosaic.


In [1]:
import rasterio
from pyproj import Transformer
from pathlib import Path
import numpy as np

# Resolve project root (notebooks/ is parallel to map/)
project_root = Path.cwd().parent
raster_path = project_root / "map" / "2017" / "alphaearth" / "brahmanbaria_alphaearth_2017_mosaic.tif"

with rasterio.open(raster_path) as src:
    src_crs = src.crs
    if src_crs is None:
        raise ValueError("Raster CRS is undefined")

    nodata = src.nodata
    if nodata is None:
        nodata = 0.0

    # Find a random valid pixel using band 1
    band1 = src.read(1)
    valid_mask = np.isfinite(band1) & (band1 != nodata)
    if not valid_mask.any():
        raise ValueError("No valid pixels found (all nodata).")

    rows, cols = np.where(valid_mask)
    idx = np.random.randint(0, len(rows))
    row, col = int(rows[idx]), int(cols[idx])

    x, y = src.xy(row, col)
    if src_crs.to_string() != "EPSG:4326":
        transformer = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
        lon, lat = transformer.transform(x, y)
    else:
        lon, lat = x, y

    # Read all bands at the chosen pixel
    window = ((row, row + 1), (col, col + 1))
    sample = src.read(window=window).reshape(src.count)

print(f"Random valid pixel row={row}, col={col}")
print(f"Lon, Lat: {lon:.6f}, {lat:.6f}")
print(f"Bands: {sample.shape[0]}")
for i, v in enumerate(sample.astype(np.float64), start=1):
    print(f"Band {i:02d}: {v:.16f}")


Random valid pixel row=10343, col=2034
Lon, Lat: 90.685153, 23.515963
Bands: 64
Band 01: 0.0711111128330231
Band 02: -0.0222068428993225
Band 03: 0.0711111128330231
Band 04: 0.0271280277520418
Band 05: -0.0246059205383062
Band 06: 0.1929104179143906
Band 07: 0.1190926581621170
Band 08: -0.1476970463991165
Band 09: -0.0888273715972900
Band 10: 0.2844444513320923
Band 11: 0.0517339482903481
Band 12: 0.0177777782082558
Band 13: -0.1860822737216949
Band 14: -0.1137408688664436
Band 15: 0.0074432911351323
Band 16: 0.2214532941579819
Band 17: -0.1998615860939026
Band 18: 0.0448442921042442
Band 19: -0.1663360297679901
Band 20: 0.0297731645405293
Band 21: -0.2844444513320923
Band 22: 0.2679584920406342
Band 23: -0.1358861923217773
Band 24: 0.1137408688664436
Band 25: 0.0448442921042442
Band 26: 0.1417300999164581
Band 27: 0.1358861923217773
Band 28: 0.0039369473233819
Band 29: -0.0797231867909431
Band 30: -0.0138408308848739
Band 31: 0.0271280277520418
Band 32: 0.0629911571741104
Band 33: -0.